In [81]:

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
import numpy as np
from utils.Preprocess import Preprocess
import os
import pandas as pd



In [5]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

# 전처리 모듈
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'user_dic.tsv'))

# ① 학습용 말뭉치 데이터를 불러옴
import pickle
with open('tsv_ner.pkl','rb') as f:
    tsv_ner = pickle.load(f)


In [6]:
tsv_ner

[[('1', '[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', 'NNG', 'B_PRODUCT'),
  ('2', '얼마', 'NNG', '0'),
  ('3', '?', 'SF', '0')],
 [('1', '[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', 'NNG', 'B_PRODUCT'),
  ('2', '얼마죠?', 'NA', '0')],
 [('1', '[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', 'NNG', 'B_PRODUCT'),
  ('2', '얼마', 'NNG', '0'),
  ('3', '냐', 'NNG', '0'),
  ('4', '?', 'SF', '0')],
 [('1', '[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', 'NNG', 'B_PRODUCT'),
  ('2', '얼마', 'NNG', '0'),
  ('3', '야', 'JKV', '0'),
  ('4', '?', 'SF', '0')],
 [('1', '[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', 'NNG', 'B_PRODUCT'),
  ('2', '얼마', 'NNG', '0'),
  ('3', '에', 'JKB', '0'),
  ('4', '요', 'JX', '0'),
  ('5', '?', 'SF', '0')],
 [('1', '[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', 'NNG', 'B_PRODUCT'),
  ('2', '얼마', 'NNG', '0'),
  ('3', '정도', 'NNG', '0'),
  ('4', '하', 'VX', '0'),
  ('5', '아요', 'EF', '0'),
  ('6', '?', 'SF', '0')],
 [('1', '[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', 'NNG', 'B_PRODUCT'),
  ('2', '얼마', 'NNG', '0'),
  ('3', '주', 'VX', '0'),
  ('4', '면', 'EC',

In [7]:
# ② 위에서 불러온 말뭉치 데이터에서 단어(w[1])와 BIO 태그(w[3])만 불러와 학습용 데이터셋 생성

sentences, tags = [], []
for t in tsv_ner:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    
    sentences.append(sentence)
    tags.append(bio_tag)



In [8]:
sentences

[['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마죠?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마', '냐', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마', '야', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마', '에', '요', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마', '정도', '하', '아요', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마', '주', '면', '되', '요', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마', '드리', '면', '되', '나요', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마나', '드리', '면', '되', '나요', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '가격', '이', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '가격'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '가격', '이', '얼마', '정도', '하', '아요', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '가격', '이', '어떻', '게', '되', '나', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '가격', '이', '어떻', '게', '되', '요', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '가격', '이', '어떻', '게', '되', '냐', '?'],
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '가격', '

In [9]:
tags

[['B_PRODUCT', '0', '0'],
 ['B_PRODUCT', '0'],
 ['B_PRODUCT', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0'],
 ['B_PRODUCT', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0'],
 ['B_PRODUCT', '0', '0', '0', '0']

In [10]:
print("샘플 크기 : \n", len(sentences))
print("0번 째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번 째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 111600
0번 째 샘플 단어 시퀀스 : 
 ['[위글위글X토앤토] 플립플랍 ZEROVITY - Cream', '얼마', '?']
0번 째 샘플 bio 태그 : 
 ['B_PRODUCT', '0', '0']
샘플 단어 시퀀스 최대 길이 : 8
샘플 단어 시퀀스 평균 길이 : 5.983870967741935


In [11]:
# ③ 토크나이저 정의
# 단어 시퀀스의 경우 Preprocess 객체에서 생성하기 때문에 예제에서는 BIO 태그용 Tokenizer 객체 생성
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)  # 나중에 texts_to_sequences() 등을 수행하기 전에 꼭


In [12]:
# 우선 확인
p.word_index

{'OOV': 1,
 '하': 2,
 '이': 3,
 '는': 4,
 '네': 5,
 '은': 6,
 '아': 7,
 '어요': 8,
 '요': 9,
 '반품': 10,
 '시': 11,
 '원': 12,
 '되': 13,
 '사이즈': 14,
 '있': 15,
 '가': 16,
 '게': 17,
 '얼마': 18,
 '에': 19,
 '나요': 20,
 '주문': 21,
 '만': 22,
 'ㅂ니다': 23,
 'ㄴ': 24,
 '주': 25,
 '았': 26,
 '고': 27,
 '색상': 28,
 '을': 29,
 '예': 30,
 '드리': 31,
 '는데': 32,
 '거': 33,
 '이것': 34,
 '좀': 35,
 '에요': 36,
 'ㄴ가요': 37,
 '가격': 38,
 '아요': 39,
 '죠': 40,
 '면': 41,
 '안': 42,
 '겠': 43,
 '것': 44,
 '으로': 45,
 'ㄹ': 46,
 '그': 47,
 '로': 48,
 '어떻': 49,
 '아서': 50,
 '그럼': 51,
 '어': 52,
 '도': 53,
 '상품': 54,
 '습니다': 55,
 '는데요': 56,
 '지금': 57,
 '없': 58,
 '취소': 59,
 '할인': 60,
 '맞': 61,
 '오': 62,
 '제가': 63,
 '를': 64,
 '아니': 65,
 '옷': 66,
 '나오': 67,
 '그러면': 68,
 '었': 69,
 '받': 70,
 '려고요': 71,
 '부탁': 72,
 '뭐': 73,
 'ㄹ게요': 74,
 '어떤': 75,
 '같': 76,
 '그것': 77,
 '크': 78,
 '보': 79,
 '배': 80,
 '제품': 81,
 '확인': 82,
 '수': 83,
 '님': 84,
 '가방': 85,
 '한': 86,
 'ㄹ까요': 87,
 'ㄹ려고': 88,
 '고객': 89,
 '사': 90,
 '아야': 91,
 '그냥': 92,
 '네요': 93,
 'ㄴ데': 94,
 '려고': 95,
 '

In [13]:
tag_tokenizer.word_index

{'0': 1, 'B_PRODUCT': 2}

In [14]:
# 단어사전 및 태그 사전 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

BIO 태그 사전 크기 : 3
단어 사전 크기 : 2927


In [15]:
# 학습용 단어 시퀀스 생성
x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
y_train = tag_tokenizer.texts_to_sequences(tags)

index_to_ner = tag_tokenizer.index_word
index_to_ner[0] = 'PAD'

In [16]:
len(x_train)

111600

In [17]:
x_train[:10]

[[1, 18, 1],
 [1, 1],
 [1, 18, 1, 1],
 [1, 18, 644, 1],
 [1, 18, 19, 9, 1],
 [1, 18, 139, 2, 39, 1],
 [1, 18, 25, 41, 13, 9, 1],
 [1, 18, 31, 41, 13, 20, 1],
 [1, 330, 31, 41, 13, 20, 1],
 [1, 38, 3, 1]]

In [18]:
y_train[:10]

[[2, 1, 1],
 [2, 1],
 [2, 1, 1, 1],
 [2, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1]]

In [19]:
index_to_ner   # 0: 'PAD'  가 추가되었다

{1: '0', 2: 'B_PRODUCT', 0: 'PAD'}

In [20]:
# ⑤ 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)


In [21]:
x_train,y_train

(array([[ 1, 18,  1, ...,  0,  0,  0],
        [ 1,  1,  0, ...,  0,  0,  0],
        [ 1, 18,  1, ...,  0,  0,  0],
        ...,
        [ 1, 99,  6, ...,  0,  0,  0],
        [ 1, 78, 24, ...,  0,  0,  0],
        [ 1, 99,  6, ...,  0,  0,  0]]),
 array([[2, 1, 1, ..., 0, 0, 0],
        [2, 1, 0, ..., 0, 0, 0],
        [2, 1, 1, ..., 0, 0, 0],
        ...,
        [2, 1, 1, ..., 0, 0, 0],
        [2, 1, 1, ..., 0, 0, 0],
        [2, 1, 1, ..., 0, 0, 0]]))

In [22]:
x_train.shape, y_train.shape

((111600, 40), (111600, 40))

In [27]:
# # ⑥ 학습데이터와 테스트 데이터를 7:3의 비율로 분리 ( 한 번 만 실 행)
# x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
#                                                     test_size=.3,
#                                                     random_state=1234)


In [28]:
len(x_train), len(x_test), len(y_train), len(y_test)

(78120, 33480, 78120, 33480)

In [29]:
y_train

array([[2, 1, 1, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0],
       ...,
       [2, 1, 1, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0]])

In [30]:
tag_size

3

In [31]:
# 출력 데이터를 one-hot encoding
# # ★ 한번만 실행할것!
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)  # tag_size. BIO 태그 사전 크기, 
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)    # 현재 tag_size 는 10 이다

In [32]:
y_train.shape , y_test.shape 
# y array[3,1,1,...,0,0,0] ... 에서 3 을 one-hot encoding
# 1 을 one-hot encoding  .....
# 하나하나 한것

((78120, 40, 3), (33480, 40, 3))

In [33]:
print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (78120, 40)
학습 샘플 레이블 형상 :  (78120, 40, 3)
테스트 샘플 시퀀스 형상 :  (33480, 40)
테스트 샘플 레이블 형상 :  (33480, 40, 3)


In [34]:
# ⑦ 모델 정의 (Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])




In [35]:
model_file = os.path.join(r'./models/ner', 'ner_model.h5')
model_file  

'./models/ner\\ner_model.h5'

In [36]:
# 학습하기
# model.fit(x_train,y_train, batch_size = 128, epochs=10)

In [37]:
# print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
# model.save(os.path.join('./models/ner', 'ner_model.h5'))  # 학습 완료된 모델 저장

In [82]:
# 의도 분류 모델 불러오기
model_name=os.path.join('./models/ner', 'ner_model.h5')
model = load_model(model_name)

In [57]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):  # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
    result = []
    for sequence in sequences:  # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence:  # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax(pred)  # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))  # 'PAD'는 'O'로 변경
        result.append(temp)
    return result

In [58]:
from seqeval.metrics import f1_score, classification_report

In [83]:
# 테스트 데이터셋의 NER 예측
# F1 score 를 게산하기 위해 모델의 predict() 함수를 통해 테스트용 데이터 셋의 결과를 예측합니다.
# 해당 함수의 입력은 시퀀스 번호로 인코딩 된 테스트 단어 시퀀스 (ndarray 배열)을 사용
# 해당 함수의 결과는 예측된 NER 태그 정보가 담긴 ndarray 배열을 리턴
y_predicted = model.predict(x_test)

In [84]:
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test)    # 실제 NER

In [85]:
# F1 평가 결과
# seqeval.metrics 모듈의 classification+_report 함수를 통해 NER 태그 별로 계산된 precision, recall 그리고 f1_score 값을 출력

print(classification_report(test_tags, pred_tags))

# f1_score() 함수 만으로 f1 score 값만 출력 가능
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

              precision    recall  f1-score   support

    _PRODUCT       1.00      1.00      1.00     33480

   micro avg       1.00      1.00      1.00     33480
   macro avg       1.00      1.00      1.00     33480
weighted avg       1.00      1.00      1.00     33480

F1-score: 100.0%


In [86]:
# 질의어 개체명 인식할.. 질의어
query = 'A-shirt 무슨 색 있어?'
pos = p.pos(query)
pos

[('A', 'SL'),
 ('-', 'SS'),
 ('shirt', 'SL'),
 ('무슨', 'MM'),
 ('색', 'NNG'),
 ('있', 'VX'),
 ('어', 'EF'),
 ('?', 'SF')]

In [87]:
keywords = p.get_keywords(pos, without_tag=True)
keywords

['A', 'shirt', '무슨', '색', '있']

In [88]:
sequences = [p.get_wordidx_sequence(keywords)]
sequences

[[1, 1, 203, 120, 15]]

In [89]:
# 패딩처리
max_len = 40
padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding="post", value=0, maxlen=max_len)
padded_seqs

array([[  1,   1, 203, 120,  15,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0]])

In [90]:
# 문장하나
# 첫번째 batch
padded_seqs[0]

array([  1,   1, 203, 120,  15,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0])

In [91]:
# 예측하기
predict = model.predict(padded_seqs)
predict

array([[[5.47974176e-14, 3.33628734e-08, 1.00000000e+00],
        [9.03181224e-21, 1.00000000e+00, 3.46965767e-09],
        [1.18406296e-25, 1.00000000e+00, 2.93520824e-15],
        [1.93093329e-25, 1.00000000e+00, 1.08419636e-19],
        [1.15023276e-24, 1.00000000e+00, 9.12871121e-25],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.03784102e-01, 3.69609714e-01, 3.26606184e-01],
        [3.037

In [93]:
predict_class = tf.math.argmax(predict, axis=-1)
predict_class

<tf.Tensor: shape=(1, 40), dtype=int64, numpy=
array([[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int64)>

In [94]:
tags = [index_to_ner[i] for i in predict_class.numpy()[0]]
print(tags)

['B_PRODUCT', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


In [95]:
# 개체명 클래스 예측 결과
predicts = list(zip(keywords, tags))
predicts

[('A', 'B_PRODUCT'), ('shirt', '0'), ('무슨', '0'), ('색', '0'), ('있', '0')]

In [96]:
# 예측한 tag 들
tags = []
for tag_idx in predict_class.numpy()[0]:
    if tag_idx == 1: continue   # O 태그 제외
    tags.append(index_to_ner[tag_idx])

tags

['B_PRODUCT']

In [97]:
tsv_ner[0][0][1]

'[위글위글X토앤토] 플립플랍 ZEROVITY - Cream'

In [98]:
p.pos(tsv_ner[0][0][1])

[('[', 'SS'),
 ('위', 'NNG'),
 ('글', 'NNG'),
 ('위', 'NNG'),
 ('글', 'NNG'),
 ('X', 'SL'),
 ('토', 'NNG'),
 ('앤', 'NNP'),
 ('토', 'NNG'),
 (']', 'SS'),
 ('플립플랍', 'NA'),
 ('ZEROVITY', 'SL'),
 ('-', 'SS'),
 ('Cream', 'SL')]